In [1]:
import sys
sys.path.append('../../../')
sys.path.append('../../')
sys.path.append('../')
sys.path.append('./')
import nltk
import jactorch
nltk.download('punkt_tab')
#export GRB_LICENSE_FILE=$HOME/gurobi.lic
from domiknows.sensor.pytorch import EdgeSensor, ModuleLearner
from domiknows.sensor.pytorch.sensors import ReaderSensor, FunctionalSensor, FunctionalReaderSensor
from domiknows.program.lossprogram import InferenceProgram
from domiknows.program.model.pytorch import SolverModel
from preprocess import preprocess_dataset
from graph import create_graph
from pathlib import Path
from modules import ResNetPatcher, DummyLinearLearner
import argparse, torch, logging

sys.argv = ['notebook', "--dummy"]
parser = argparse.ArgumentParser(description="Logic-guided VQA training / evaluation")
parser.add_argument("--train-size", type=int, default=10,help="Number of training examples to sample (default: use full set)")
parser.add_argument("--test-size", type=int, default=10,help="Number of test examples to sample (default: use full set)")
parser.add_argument("--epochs", type=int, default=4,help="Number of training epochs")
parser.add_argument("--lr", "--learning-rate", type=float, default=1e-6,help="Learning rate")
parser.add_argument("--batch-size", type=int, default=1,help="Mini-batch size")
parser.add_argument("--eval-only", action="store_true",help="Skip training; just load a checkpoint and evaluate")
parser.add_argument("--dummy", action="store_true",help="Use the lightweight dummy configuration")
parser.add_argument("--tnorm", choices=["G", "P", "L"], default="G",help="T-norm used inside InferenceProgram")
args=parser.parse_args()

CACHE_DIR      = Path("dataset_cache2")
CACHE_DIR.mkdir(exist_ok=True)
NUM_INSTANCES  = 10
device = "cpu"

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /localscratch/premsrit/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/localscratch2/premsrit/DomiKnowS/test_regr/Clever/../../domiknows/solver/ilpOntSolver.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Log file for dataNode is in: /localscratch2/premsrit/DomiKnowS/test_regr/Clever/logs/datanode.log


[nltk_data] Downloading package punkt to
[nltk_data]     /localscratch/premsrit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /localscratch/premsrit/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
dataset = preprocess_dataset(args,NUM_INSTANCES,CACHE_DIR)

re-loaded dataset_cache2/instance_0.pkl
re-loaded dataset_cache2/instance_1.pkl
re-loaded dataset_cache2/instance_2.pkl
re-loaded dataset_cache2/instance_3.pkl
re-loaded dataset_cache2/instance_4.pkl
re-loaded dataset_cache2/instance_5.pkl
re-loaded dataset_cache2/instance_6.pkl
re-loaded dataset_cache2/instance_7.pkl
re-loaded dataset_cache2/instance_8.pkl
re-loaded dataset_cache2/instance_9.pkl
Dataset length: 10


In [3]:
image=dataset[0]

In [4]:
from modules import ResnetLEFT, LEFTObjectEMB, LEFTRelationEMB

resnet = ResnetLEFT("cpu")
object_feature_extraction = LEFTObjectEMB(device="cpu")
relation_feature_extraction = LEFTRelationEMB(device="cpu")

In [5]:
scene_emb = resnet(99,image["pil_image"])